# <span style="color:#72A98F">Location from Timestamp Notebook</span>

Hello Person! In this notebook I will be using the *[LLAT](../../Data/Location/LIGHT-1_LLA_April.csv)* (Latitude Longitude Altitude Time) file to find the location at a given **Event Timestamp**.

## <span style="color:#72A98F">Importing libraries and giving constant values</span>

In [295]:
import numpy as np
import pandas as pd
from astropy.time import Time
import raadpy as rp
import os
from datetime import datetime

TimeName = 'Time (ModJDate)' #The Time Column Title

In [ ]:
# def Location_from_Timestamp(TSA):

## <span style="color:#72A98F">Finding Timestamp location for a specific value</span>

In [2]:
# GivenTimestamp = E.get_mjd()
# A remake of this line above will be used to actually get the timestamp ^
# But for now, we will name a value that exists in the range of april.
GivenTimestamp = 59639

#If the timestamp lies somewhere in March this will use the March file!
if Time(59639.00000000, format='mjd') <= Time(GivenTimestamp, format='mjd') <= Time(59670.00000000, format='mjd'):
    print("This time lies in March!")
    LocationFile = '../../Data/Location/LIGHT-1_LLA_March.csv'
    LLATData = pd.read_csv(LocationFile)
    
#If the timestamp lies somewhere in April this will use the April file!
elif Time(59670.00000000, format='mjd') <= Time(GivenTimestamp, format='mjd') <= Time(59700.00000000, format='mjd'):
    print("This time lies in April!")
    LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
    LLATData = pd.read_csv(LocationFile)
    
#If the timestamp reached does not exist in the range useable, this will skip that specific value only.
else:
    print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")

#Read the time column only
TData = LLATData["Time (ModJDate)"]

#Count the rows and look through them.
for row in range(len(TData)):

    #
    if GivenTimestamp == TData[row]:
        MaximumData = LLATData.T[row]
        MinimumData = LLATData.T[row]
        break
    
    if TData[row] >= GivenTimestamp:
        MaximumData = LLATData.T[row]
        MinimumData = LLATData.T[row-1]
        break

TimeRange = [MinimumData[0], MaximumData[0]]
LongRange = [MinimumData[2], MaximumData[2]]
LatRange  = [MinimumData[1], MaximumData[1]]
longitude2 = np.interp(GivenTimestamp, TimeRange, LongRange)
latitude2 = np.interp(GivenTimestamp, TimeRange, LatRange)

TS = [] #For the values to be added together.
TS.append([GivenTimestamp, longitude2, latitude2])
TS

This time lies in March!


[[59639, -157.984, -31.383]]

## <span style="color:#72A98F">Finding Timestamp Locations for an Array of Timestamps

In [3]:
TSA  = [59621,59671]
TSL = []

for ts in TSA:

    #If the timestamp lies somewhere in March this will use the March file!
    if Time(59639.000000000, format='mjd') <= Time(ts, format='mjd') <= Time(59670.00000000, format='mjd'):
        LocationFile = '../../Data/Location/LIGHT-1_LLA_March.csv'
        LLATData = pd.read_csv(LocationFile)
        
    
    #If the timestamp lies somewhere in April this will use the April file!
    elif Time(59670.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59700.00000000, format='mjd'):
        LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
        LLATData = pd.read_csv(LocationFile)
    
    #If the timestamp reached does not exist in the range useable, this will skip that specific value only.
    else:
        print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")
        continue #Skips the values not in the range!

    #Telling it to take the time column
    TData = LLATData["Time (ModJDate)"]

    #Start the loop that looks through the file.
    for row in range(len(TData)):

        #If the timestamp exists in the data file, use the location already given, and leave the loop. (Condition 1)
        if ts == TData[row]:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row]
            break
        
        #If the timestamp does not exist in the file, then take the values before and after it, then leave the loop. (Condition 2)
        if TData[row] >= ts:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row-1]
            break
    
    #If Condition 1 is met, this part will not affect any values.
    #Otherwise, if Condition 2 is met, this part will interpolate the values linearly. 
    #(Hopefully using different interpolation methods later!)
    TimeRange = [MinimumData[0], MaximumData[0]]
    LongRange = [MinimumData[2], MaximumData[2]]
    LatRange  = [MinimumData[1], MaximumData[1]]
    longitude = np.interp(ts, TimeRange, LongRange)
    latitude = np.interp(ts, TimeRange, LatRange)

    #Append the values into an array.
    TSL.append([ts,longitude,latitude])

print(TSL)

The time value given does not exist in the range from March to April!
For reference, the time in mjd should be between 59639.0 and 59700.0


KeyError: 'Time (ModJDate)'

## <span style="color:#72A98F">*Ignore what's below this point*</span>

In [3]:
#NOTES

# time = E.get_mjd()

# def f(EV:rp.event):
#     time = EV.get_mjd()

# k = [(i) for i in arr if "June" in i ]
# k

# def g(a):
#     return a+

# import numpy as np
# x = np.random.rand()
# y = np.random.rand(10)
# np.where(x < y)[0]

In [317]:
LocationFolder = '../../Data/Location/'
arr = os.listdir(LocationFolder)

months = ['March','April','May','June','July','August','September','October','November']
FilesFound = []

for m in months:
    k = [(i) for i in arr if m in i ]
    FilesFound.append(k)

FileData={}

for i in range(len(months)):
    if len(FilesFound[i]) != 0:
        print(months[i] + " file is found and opened!")
        FileData[months[i]] = pd.read_csv('../../Data/Location/'+FilesFound[i][0])
        
    else:
        print(months[i] + " file is NOT found!")

March file is found and opened!
April file is found and opened!
May file is NOT found!
June file is NOT found!
July file is NOT found!
August file is NOT found!
September file is NOT found!
October file is NOT found!
November file is NOT found!


In [308]:
TSA = np.random.uniform(low=59639.000000000, high=59700.00000000, size=30)
print(TSA)

[59664.65500886 59661.51617185 59681.13897162 59686.16187424
 59663.54948367 59689.57740083 59655.11785478 59639.03369846
 59670.79812471 59679.28029505 59644.42487347 59691.21528367
 59648.10610441 59655.26333064 59671.14526877 59691.2785713
 59677.9729217  59673.52994349 59639.58184005 59698.4451969
 59680.12852823 59664.26764047 59689.1431089  59699.28660119
 59656.41511056 59680.1588639  59644.83661097 59645.06577001
 59645.41856605 59672.04286013]


In [338]:
Values = {}

for i in range(len(months)):

    if len(FilesFound[i]) != 0:
        MonthStart = datetime.strptime(months[i], '%B').month
        MonthEnd   = datetime.strptime(months[i], '%B').month + 1

        Values[months[i]] = np.where(np.logical_and(TSA <= Time('2022-'+str(MonthEnd)+'-01').mjd, TSA >= Time('2022-'+str(MonthStart)+'-01').mjd))[0]

    else:
        continue

print(Values['April'][15])

29


In [355]:
TSL=[]

print('Using months found to find values!')

for i in range(len(FilesFound)):

    print(i,months[i],len(FilesFound[i]),len(FilesFound[i]) != 0)

    if len(FilesFound[i]) != 0:
        for j in Values[months[i]]:

            print(j,Values[months[i]],months[i])
            ts = TSA[j]
            print(ts)

            ResultMin = FileData[months[i]][FileData[months[i]][TimeName] <= ts].nlargest(1, TimeName)
            ResultMax = FileData[months[i]][FileData[months[i]][TimeName] >= ts].nsmallest(1, TimeName)

            print(float(ResultMin.iloc[:,(0)]))
            
            TimeRange = [float(ResultMin.iloc[:,(0)]), float(ResultMax.iloc[:,(0)])]
            LongRange = [float(ResultMin.iloc[:,(2)]), float(ResultMax.iloc[:,(2)])]
            LatRange  = [float(ResultMin.iloc[:,(1)]), float(ResultMax.iloc[:,(1)])]
            longitude = np.interp(ts, TimeRange, LongRange)
            latitude = np.interp(ts, TimeRange, LatRange)
            TSL.append([ts,longitude,latitude])
    
    else:
        continue

print(TSL)

    
    

Using months found to find values!
0 March 1 True
0 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59664.655008857204
1 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59661.516171852796
4 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59663.54948367149
6 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59655.117854783704
7 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59639.03369846455
10 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59644.42487346503
12 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59648.10610440769
13 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59655.263330642396
18 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59639.581840053805
21 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59664.267640474536
24 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59656.41511055544
26 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59644.836610974264
27 [ 0  1  4  6  7 10 12 13 18 21 24 26 27 28] March
59645.065770008216
28 [ 0  1  4  6  7 10 12

In [348]:
# ResultMin = FileData[months[i]][FileData[months[i]][TimeName] <= ts].nlargest(1, TimeName)
print(len(FilesFound[1]))

1


In [93]:
for i in MarchValues:
    ts = TSA[i]
print(ts)

59671.86659570189


In [24]:
# TSA  = [59621,59671]

TSL = []
np.where(np.logical_and(TSA <= 59700.00000000, TSA >= 59639.000000000))[0]

for ts in TSA:

    if Time('2022-03-01') <= Time(ts, format='mjd') <= Time('2022-03-01'):
        LLATData = FileData['March']
        
    
    elif Time(59670.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59700.00000000, format='mjd'):
        LLATData = FileData['April']
    
    else:
        print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")
        continue

    TData = LLATData["Time (ModJDate)"]

    for row in range(len(TData)):

        if ts == TData[row]:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row]
            break
        
        if TData[row] >= ts:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row-1]
            break
    
    TimeRange = [MinimumData[0], MaximumData[0]]
    LongRange = [MinimumData[2], MaximumData[2]]
    LatRange  = [MinimumData[1], MaximumData[1]]
    longitude = np.interp(ts, TimeRange, LongRange)
    latitude = np.interp(ts, TimeRange, LatRange)
    TSL.append([ts,longitude,latitude])

print(TSL)

NameError: name 'TSA' is not defined